In [1]:
import json
import ast
import pandas as pd
import numpy as np
import gzip

## Read json dataset

In [2]:
file_name = "amazon/Movies_and_TV_5.json.gz"
seq_len = 10

In [3]:
def parse(path):
    g = gzip.open(path, 'rb')
    for l in g:
        yield json.loads(l)

def getDF(path):
    i = 0
    df = {}
    for d in parse(path):
        df[i] = d
        i += 1
    return pd.DataFrame.from_dict(df, orient='index')

In [4]:
data_table = getDF(file_name)
print(data_table.shape)

(3410019, 12)


In [6]:
data_table.head()

,overall,verified,reviewTime,reviewerID,asin,style,reviewerName,reviewText,summary,unixReviewTime,vote,image
0,5.0,True,"11 9, 2012",A2M1CU2IRZG0K9,0005089549,{'Format:': ' VHS Tape'},Terri,So sorry I didn't purchase this years ago when...,Amazing!,1352419200,NaN,NaN
1,5.0,True,"12 30, 2011",AFTUJYISOFHY6,0005089549,{'Format:': ' VHS Tape'},Melissa D. Abercrombie,Believe me when I tell you that you will recei...,Great Gospel VHS of the Cathedrals!,1325203200,NaN,NaN
2,5.0,True,"04 21, 2005",A3JVF9Y53BEOGC,000503860X,{'Format:': ' DVD'},Anthony Thompson,"I have seen X live many times, both in the ear...",A great document of a great band,1114041600,11,NaN
3,5.0,True,"04 6, 2005",A12VPEOEZS1KTC,000503860X,{'Format:': ' DVD'},JadeRain,"I was so excited for this! Finally, a live co...",YES!! X LIVE!!,1112745600,5,NaN
4,5.0,True,"12 3, 2010",ATLZNVLYKP9AZ,000503860X,{'Format:': ' DVD'},T. Fisher,X is one of the best punk bands ever. I don't ...,X have still got it,1291334400,5,NaN


In [5]:
# data_table.drop_duplicates(subset=['overall', 'verified', 'reviewTime', 'reviewerID', 'asin',
#        'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote'], inplace=True)
# print(data_table.shape)

In [6]:
data_table['reviewLen'] = data_table['reviewText'].str.len()

In [7]:
data_table.drop(columns=['style', 'reviewerName', 'reviewText', 'summary', 'image', 'vote'], inplace=True)

In [8]:
print(data_table.duplicated().sum())

112194


In [ ]:
data_table.head()

## Filter out user with few ratings

In [ ]:
user_cnt = data_table.groupby('reviewerID').agg(
 user_count=('overall', 'count')
)

In [ ]:
data_table1 = data_table.join(user_cnt, on='reviewerID')

In [ ]:
data_table1.drop(data_table1.loc[data_table1['user_count']<=4].index, inplace=True)

In [ ]:
print(data_table1.shape)
data_table1.head()

In [ ]:
data_table1.drop(columns=['user_count'], inplace=True)

In [9]:
data_table1['reviewTimeDayOfWeek'] = pd.to_datetime(data_table1['unixReviewTime'], unit='s').dt.dayofweek
data_table1['reviewTimeHour'] = pd.to_datetime(data_table1['unixReviewTime'], unit='s').dt.hour
data_table1.sort_values(by=['reviewerID', 'unixReviewTime'],inplace=True)

data_table1['rownumber'] = np.arange(data_table1.shape[0])

In [ ]:
print(data_table1.shape)
data_table1.head()

## Split data into train, val, test

In [18]:
def get_train_data(x):
    return x.index[:-2]

def get_val_data(x):
    return x.index[-1 - seq_len:-1]

def get_test_data(x):
    return x.index[-seq_len:]

def get_all_data(x):
    d = {}
    d['train_idx'] = get_train_data(x)
    d['val_idx'] = get_val_data(x)
    d['test_idx'] = get_test_data(x)
    return pd.Series(d, index=['train_idx', 'val_idx', 'test_idx'])

In [ ]:
user_group = data_table1.groupby('reviewerID')
user_group.head()

In [ ]:
all_idx = user_group['reviewerID'].apply(get_all_data)
all_idx.head()

In [ ]:
train_idx = np.concatenate(all_idx.loc[:,'train_idx'])
val_idx = np.concatenate(all_idx.loc[:,'val_idx'])
test_idx = np.concatenate(all_idx.loc[:,'test_idx'])

In [ ]:
train_table0 = data_table1.loc[train_idx]
val_table0 = data_table1.loc[val_idx]
test_table0 = data_table1.loc[test_idx]

## Calculate global statistics or static table using training data only

In [11]:
def get_high_per(x):
    return (x>=4).sum()/len(x)

def get_low_per(x):
    return (x<=3).sum()/len(x)

def get_pos_reviewer(x):
    #return (x>=4).sum() > (x<=3).sum()
    if (x>=4).sum() > (x<=3).sum():
        return 'isPos'
    else:
        return 'isNeg'

def get_top_item(x):
    x_ov = x.apply(lambda x: x.split('_')[0])
    x_as = x.apply(lambda x: x.split('_')[1])
    top_idx = x_ov.sort_values(ascending=False).iloc[0:3].index
    top_item = x_as[top_idx].tolist()
    return top_item


In [12]:
static_table = train_table0.groupby('reviewerID').agg(
    avg_rating=('overall', 'mean'),
    as_count=('overall', 'count'),
    low_per=('overall', lambda x: get_low_per(x)),
    high_per=('overall', lambda x: get_high_per(x)),
    pos_reviewer=('overall', lambda x: get_pos_reviewer(x)),
    # top_item=('o_a', lambda x: get_top_item(x)),
)

In [ ]:
static_table.head()

In [ ]:
static_table.isnull().any()

## Global default: reviewer rating

In [ ]:
df_avg_rating = train_table0['overall'].mean()
print(df_avg_rating)

## Asin rating

In [ ]:
asin_rating = train_table0.groupby('asin').agg(
 asin_rating=('overall', 'mean'),
 asin_cnt=('overall', 'count'),
)


In [ ]:
asin_rating.head()

## Global default: asin rating

In [ ]:
df_asin_rating = train_table0['overall'].mean()
print(df_asin_rating)

## Join tables

In [ ]:
train_table1 = train_table0.join(static_table, on='reviewerID')
val_table1 = val_table0.join(static_table, on='reviewerID')
test_table1 = test_table0.join(static_table, on='reviewerID')

In [ ]:
train_table2 = train_table1.join(asin_rating, on='asin')
val_table2 = val_table1.join(asin_rating, on='asin')
test_table2 = test_table1.join(asin_rating, on='asin')

In [ ]:
print(train_table2.shape, val_table2.shape, test_table2.shape)

In [ ]:
train_table2.head()

## Filling default values

In [ ]:
print(train_table2.isnull().any())
print(val_table2.isnull().any())
print(test_table2.isnull().any())

In [ ]:
df_values = {"avg_rating": df_avg_rating, "asin_rating ": df_asin_rating}
train_table2.fillna(value=df_values, inplace=True)
#dy: what about val and test table?
val_table2.fillna(value=df_values, inplace=True)
test_table2.fillna(value=df_values, inplace=True)

In [ ]:
drop_cols = ['as_count', 'asin_cnt']
for df in [train_table2, val_table2, test_table2]:
    df.drop(columns=drop_cols, inplace=True)

In [30]:
print(train_table2.shape, val_table2.shape, test_table2.shape)

(2814911, 17) (1996181, 17) (2214747, 17)


In [31]:
train_file_name = file_name.split('.')[0] + '_train.csv'
val_file_name = file_name.split('.')[0] + '_val.csv'
test_file_name = file_name.split('.')[0] + '_test.csv'
all_file_name = file_name.split('.')[0] + '.csv'
print(train_file_name, val_file_name, test_file_name, all_file_name)

amazon/Movies_and_TV_5_train.csv amazon/Movies_and_TV_5_val.csv amazon/Movies_and_TV_5_test.csv amazon/Movies_and_TV_5.csv


In [32]:
train_table2.to_csv(train_file_name, index=False)
val_table2.to_csv(val_file_name, index=False)
test_table2.to_csv(test_file_name, index=False)